### 处理数据 因为数据的名称不是统一的格式 所以先利用os得到文件名称，之后进行读取

In [2]:
import os
import sys
import numpy as np
import gensim
from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier

def listtext(path):
    list_name = []
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            listdir(file_path, list_name)
        else:
            list_name.append(file_path)
    list_text = []
    for files in list_name:
        file = open(files,'r',encoding = 'UTF-8')
        list_text.append(file.readlines())
    return list_text

def get_dataset(pos_reviews, neg_reviews, unsup_reviews):
    y = np.concatenate((np.ones(len(pos_reviews)), 
                        np.zeros(len(neg_reviews))))
    x_train, x_test, y_train, y_test = train_test_split(
        np.concatenate((pos_reviews, neg_reviews)), y, test_size = 0.2)#train_test_split 分开训练集 测试集

    x_train = cleanText(x_train)
    x_test = cleanText(x_test)
    unsup_reviews = cleanText(unsup_reviews)
    
    def labelizedReviews(reviews, label_type):
        LabeledSentence = gensim.models.doc2vec.LabeledSentence
        labelized = []
        for i, v in enumerate(reviews):
            label = '%s_%s'%(label_type, i)
            labelized.append(LabeledSentence(v,[label]))
        return labelized
    x_train = labelizedReviews(x_train, 'TRAIN')
    x_test = labelizedReviews(x_test, 'TEST')
    unsup_reviews = labelizedReviews(unsup_reviews, 'UNSUP')
    return x_train, x_test, unsup_reviews, y_train, y_test
    

def getVecs(model, corpus, size):
    vecs = [np.array(model.docvecs[z.tags[0]]).reshape((1,size))
            for z in corpus]
    return np.concatenate(vecs)

def trian(x_train, x_test, unsup_reviews, size=100, epoch_num=10):
    all_review = (x_train+x_test+unsup_reviews)
    model_dm = Doc2Vec(all_review, min_count=1, window=10, size=size,
                                    negative =5, workers=3)
    '''
    model_dbow = Doc2Vec(all_review, min_count=1, window=10, size= size,
                                    negative=5, dm=0,
                                       workers=3)
    '''
    return model_dm
def get_vectors(model_dm):
    train_vecs_dm = getVecs(model_dm, x_train, size)
    #train_vecs_dbow = getVecs(model_dbow, x_train, size)
    #train_vecs = np.hstack((train_vecs_dm, train_vecs_dbow))
    test_vecs_dm = getVecs(model_dm, x_test, size)
    #test_vecs_dbow = getVecs(model_dbow, x_test, size)
    #test_vecs = np.hstack((test_vecs_dm, test_vecs_dbow))
    return train_vecs_dm, test_vecs_dm

def Classifier(train_vecs, y_train, test_vecs, y_test):  
    lr = SGDClassifier(loss = 'log', penalty = 'l1')
    lr.fit(train_vecs, y_train)
    print('Test Accuracy: %.2f'%lr.score(test_vecs, y_test))
    return lr

def ROC_curve(test_vecs, lr, y_test):
    from sklearn.metrics import roc_curve, auc
    import matplotlib.pyplot as plt
    pred_probas = lr.predict_proba(test_vecs)[:,1]

    fpr, tpr, _ = roc_curve(y_test, pred_probas)
    roc_auc = auc(fpr, tpr)
    print(roc_auc)
    plt.plot(fpr, tpr, label = 'area = %.2f'%roc_auc)
    plt.plot([0,1], [0,1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.show()
    
current_path = os.getcwd()

path_neg = current_path + '/acllmdb/aclImdb/train/neg'
path_pos = current_path + '/acllmdb/aclImdb/train/pos'
path_unsup = current_path + '/acllmdb/aclImdb/train/unsup'
#path_neg = 'F:/acllmdb/aclImdb/train/neg'
#path_pos = 'F:/acllmdb/aclImdb/train/pos'
#path_unsup = 'F:/acllmdb/aclImdb/train/unsup'

pos_reviews = listtext(path_pos)
neg_reviews = listtext(path_neg)
unsup_reviews = listtext(path_unsup)

model_dm = trian(x_train, x_test, unsup_reviews1,
                             size, epoch_num)
size , epoch_num = 400,10
x_train, x_test, unsup_reviews1, y_train, y_test = get_dataset(pos_reviews, neg_reviews, unsup_reviews)
train_vecs, test_vecs = get_vectors(model_dm)
lr = Classifier(train_vecs, y_train, test_vecs, y_test)

ROC_curve(test_vecs, lr, y_test)

NameError: name 'x_train' is not defined